In [1]:
import datacube
from datacube.utils.cog import write_cog
from datacube.testutils.io import rio_slurp_xarray
import pandas as pd
import numpy as np
import xarray as xr
from scipy.signal import savgol_filter, find_peaks
from scipy.interpolate import CubicSpline
from dask.diagnostics import ProgressBar
import datetime

import gba_functions
import gba_pheno

import sys
#import pprint
#pprint.pprint(sys.path)

#sys.path.insert(1,'.../dev/dea-notebooks/Tools/') 
sys.path.insert(1,'/home/jovyan/dev/dea-notebooks/Tools') 

#sys.path.insert(2, '.../dev/dea-notebooks/Tools/')
print(sys.path)
from dea_tools.dask import create_local_dask_cluster
from dea_tools.datahandling import wofs_fuser
from dea_tools.spatial import xr_rasterize

['/home/jovyan/dev/dea-notebooks/GBA/GBA_Cooper', '/home/jovyan/dev/dea-notebooks/Tools', '/home/jovyan/dev/dea-notebooks/Tools', '/env/lib/python310.zip', '/env/lib/python3.10', '/env/lib/python3.10/lib-dynload', '', '/env/lib/python3.10/site-packages']


In [2]:
dc = datacube.Datacube(app='Waterhole Phenology')

In [3]:
client = create_local_dask_cluster(return_client=True)

2023-12-21 00:34:02,412 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ag7wi1x2', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/marcelo.henriquesdasilva@monash.edu/proxy/8787/status,
Dashboard: /user/marcelo.henriquesdasilva@monash.edu/proxy/8787/status,Workers: 1
Total threads: 2,Total memory: 12.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45017,Workers: 1
Dashboard: /user/marcelo.henriquesdasilva@monash.edu/proxy/8787/status,Total threads: 2
Started: Just now,Total memory: 12.21 GiB
Comm: tcp://127.0.0.1:33525,Total threads: 2
Dashboard: /user/marcelo.henriquesdasilva@monash.edu/proxy/38647/status,Memory: 12.21 GiB
Nanny: tcp://127.0.0.1:37945,


In [10]:
attribute_col = 'Waterhole'

# list of waterholes to process
#waterhole_list = ['Currareva', 'Arning']

#waterhole_list = ['Currareva', 'Arning', 'Wombundarry', 'Tanbar','Eulbertie', 'MulgaCorner', 'Marboo', 'Tarquoh', 'Ergelear', 'Widindra',
#                  'Tabbareah', 'Meringhina', 'Bogaller', 'Naccowlah', 'Goonbabina', 'Maapoo', 'Cullyamurra', 'ScrubbyCamp', 'Tirrawarra',
#                  'Kudriemitchie']

waterhole_list = ['Eulbertie']

# Path to the waterhole shapefile
#waterhole_shp = ['Shapes/Currareva/Currareva_GBA.shp', 'Shapes/Arning/Arning.shp', 'Shapes/Wombundarry/Wombundarry_GBA.shp',
#                 'Shapes/Tanbar/Tanbar_GBA.shp', 'Shapes/Eulbertie/Eulbertie.shp', 'Shapes/MulgaCorner/MulgaCorner.shp',
#                 'Shapes/Marboo/Marboo.shp', 'Shapes/Tarquoh/Tarquoh.shp', 'Shapes/Ergelear/Ergelear.shp',
#                 'Shapes/Widindra/Widindra.shp', 'Shapes/Tabbareah/Tabbareah.shp', 'Shapes/Meringhina/Meringhina.shp',
#                 'Shapes/Bogaller/Bogaller.shp', 'Shapes/Naccowlah/Naccowlah.shp', 'Shapes/Goonbabina/Goonbabinna_GBA.shp',
#                 'Shapes/Maapoo/Maapoo.shp', 'Shapes/Cullyamurra/Cullyamurra.shp', 'Shapes/ScrubbyCamp/ScrubbyCamp_GBA.shp',
#                 'Shapes/Tirrawarra/Tirrawarra.shp', 'Shapes/Kudriemitchie/Kudriemitchie.shp']

waterhole_shp = ['Shapes/Eulbertie/Eulbertie.shp']


# Path to the LiDAR derived products
#vegetation_cov = ['Rasters/Currareva/currareva_cov.tif', 'Rasters/Arning/arning_cov.tif', 'Rasters/Wombundarry/wombunderry_cov.tif',
#                  'Rasters/Tanbar/tanbar_cov.tif', 'Rasters/Eulbertie/eulbertie_cov.tif', 'Rasters/MulgaCorner/mulgacorner_cov.tif',
#                  'Rasters/Marboo/marboo_cov.tif', 'Rasters/Tarquoh/tarquoh_cov.tif', 'Rasters/Ergelear/ergelear_cov.tif',
#                  'Rasters/Widindra/widindra_cov.tif', 'Rasters/Tabbareah/tabbareah_cov.tif', 'Rasters/Meringhina/meringhina_cov.tif',
#                  'Rasters/Bogaller/bogaller_cov.tif', 'Rasters/Naccowlah/naccowla_cov.tif', 'Rasters/Goonbabina/goonbabinna_cov.tif',
#                  'Rasters/Maapoo/maapoo_cov.tif', 'Rasters/Cullyamurra/cullyamurra_cov.tif', 'Rasters/ScrubbyCamp/scrubbycamp_cov.tif',
#                  'Rasters/Tirrawarra/tirrawarra_cov.tif', 'Rasters/Kudriemitchie/kudriemitchie_cov.tif']

vegetation_cov = ['Rasters/Eulbertie/eulbertie_cov.tif']


#vegetation_avg = ['Rasters/Currareva/currareva_avg.tif', 'Rasters/Arning/arning_avg.tif', 'Rasters/Wombundarry/wombunderry_avg.tif',
#                  'Rasters/Tanbar/tanbar_avg.tif', 'Rasters/Eulbertie/eulbertie_avg.tif', 'Rasters/MulgaCorner/mulgacorner_avg.tif',
#                  'Rasters/Marboo/marboo_avg.tif', 'Rasters/Tarquoh/tarquoh_avg.tif', 'Rasters/Ergelear/ergelear_avg.tif',
#                  'Rasters/Widindra/widindra_avg.tif', 'Rasters/Tabbareah/tabbareah_avg.tif', 'Rasters/Meringhina/meringhina_avg.tif',
#                  'Rasters/Bogaller/bogaller_avg.tif', 'Rasters/Naccowlah/naccowla_avg.tif', 'Rasters/Goonbabina/goonbabinna_avg.tif',
#                  'Rasters/Maapoo/maapoo_avg.tif', 'Rasters/Cullyamurra/cullyamurra_avg.tif', 'Rasters/ScrubbyCamp/scrubbycamp_avg.tif',
#                  'Rasters/Tirrawarra/tirrawarra_avg.tif', 'Rasters/Kudriemitchie/kudriemitchie_avg.tif']

vegetation_avg = ['Rasters/Eulbertie/eulbertie_avg.tif']


# query with timeseries inteval
veg_proxy = 'NDVI'
query = {'time': ('1998-01-01','2020-12-31')
         }

In [11]:
loaded_waterholes = {}
loaded_classes = {}
for waterhole in range(len(waterhole_list)):
    print(waterhole_list[waterhole])
    
    ds, wofls, gdf = gba_functions.load_data(dc, query, waterhole_shp[waterhole], perc_good=0)
    ds_no_water, ds_cov, ds_avg, ds_water = gba_functions.prepare_data(ds, wofls, gdf, vegetation_cov[waterhole], 
                                                                       vegetation_avg[waterhole], perc_good=0)
    ds_no_water = gba_functions.chebyshev(ds_no_water, prob=0.05)
    ds_resample = ds_no_water.resample(time="16D").max('time')
    
    ds_classes, ds_mask = gba_functions.lidar_veg(ds_cov, ds_avg)
    
    #ds_resample = ds_resample.compute()
    ds_fPARs = gba_pheno.calc_fpar(ds_resample)
    
    pheno_template = pd.DataFrame(columns=['SOS_dt','SOS_doy','SOS_v','POS_dt','POS_doy','POS_v','EOS_dt','EOS_doy','EOS_v','x','y', 'Vegetation', 'LOS'])
    pheno_template_r = pd.DataFrame(columns=['SOS_dt','SOS_doy','SOS_v','POS_dt','POS_doy','POS_v','EOS_dt','EOS_doy','EOS_v','x','y', 'Vegetation', 'LOS'])
    pheno_template_p = pd.DataFrame(columns=['SOS_dt','SOS_doy','SOS_v','POS_dt','POS_doy','POS_v','EOS_dt','EOS_doy','EOS_v','x','y', 'Vegetation', 'LOS'])
    for vegetation in ds_classes.keys():
        print(vegetation)

        pheno_full, pheno_full_r, pheno_full_p = gba_pheno.phenopix(ds_fPARs, ds_classes, veg_class=vegetation, threshold=0.2)
        
        pheno_full = pheno_full.assign(Waterhole=waterhole_list[waterhole])
        pheno_full_r = pheno_full_r.assign(Waterhole=waterhole_list[waterhole])
        pheno_full_p = pheno_full_p.assign(Waterhole=waterhole_list[waterhole])
        
        pheno_template = pd.concat([pheno_template, pheno_full])
        pheno_template_r = pd.concat([pheno_template_r, pheno_full_r])
        pheno_template_p = pd.concat([pheno_template_p, pheno_full_p])
    
        client.restart()
        
    out_path = "Outputs/Phenology/"+waterhole_list[waterhole]+"_fPAR.csv"
    pheno_template.to_csv(out_path, index=False)
    
    out_path_r = "Outputs/Phenology/"+waterhole_list[waterhole]+"_fPAR_R.csv"
    pheno_template_r.to_csv(out_path_r, index=False)
    
    out_path_p = "Outputs/Phenology/"+waterhole_list[waterhole]+"_fPAR_P.csv"
    pheno_template_p.to_csv(out_path_p, index=False)

Eulbertie
Finding datasets
    ga_ls5t_ard_3
    ga_ls7e_ard_3
    ga_ls8c_ard_3
Applying fmask pixel quality/cloud mask
Returning 1505 time steps as a dask array
Dropping bands ['nbart_red', 'nbart_nir']
Dense trees


2023-12-22 03:51:07,255 - distributed.nanny - WARNING - Restarting worker


Mid-dense trees


2023-12-22 06:23:29,939 - distributed.nanny - WARNING - Restarting worker


Sparse trees


...

2023-12-23 09:29:58,178 - distributed.nanny - WARNING - Restarting worker


Isolated shrubs


2023-12-23 14:42:31,440 - distributed.nanny - WARNING - Restarting worker


Grasses and bare soil


2023-12-23 20:03:47,156 - distributed.nanny - WARNING - Restarting worker
